# Puzzle 1: Apply FFT 100 times to input

### pattern [0, 1, 0, -1]; skip first as offset; repeating digits for +ith input digit; keep only ones digit of result; 

In [63]:
import numpy as np
from copy import deepcopy

## Load input

In [178]:
with open('./input16.txt', 'r') as file:

    data = file.readlines()

#### data to array of ints

In [179]:
data = np.array([int(x) for x in data[0]])

In [184]:
number = deepcopy(data)

In [185]:
def repeat_pattern(pattern, repetitions, length):
    new_pattern = ()
    for i in pattern:
        new_pattern += (i,)*repetitions
    new_pattern = new_pattern*(length+1)
    return new_pattern[1:length+1]

In [186]:
pattern = (0, 1, 0, -1)

In [187]:
for j in range(0, 100):
    newnumber = []
    print(j)
    for i in range(1, len(number)+1):
        pat = repeat_pattern(pattern, i, len(number))
        next_digit = sum(np.multiply(number, np.asarray(pat)))
        newnumber.append(int(str(next_digit)[-1]))
    number = np.array(deepcopy(newnumber))
digits = ''
digits = digits.join([str(x) for x in number])
print('The first eight digits are {0}.'.format(digits[0:8]))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


'58672132580755468361471781172153971180956668534025024806898395488155925288156142426979913611636541827792117094231775787036499470974126114356567218702366217565639596759821380156288517532149028263228322853373468568247921766462500010768826058735470968623018426541705476270667114845528248012706612179613730718388033513894471278847986427223747047049659143419294914065804957790818993455784548745816317788327934074223670886849031794946549997215448069331456302732144969613001881743255784026893914051700990494832952063201854455300240287791607020620761641895796559555915567822908856342261079550404005391617401873850315536400623091863054131789964955458682511498'

In [191]:
print('The first eight digits are {0}.'.format(digits[0:8]))

The first eight digits are 58672132.


# Puzzle 2: Apply FFT 100 times to input with longer list and message offset

In [231]:
with open('./input16.txt', 'r') as file:

    data = file.readlines()

In [232]:
inputnumber = np.array([int(x) for x in data[0]])

In [233]:
inputnumber = np.tile(inputnumber, 10000)

In [234]:
offset = int(data[0][0:7])
number = deepcopy(inputnumber[offset:])

In [235]:
len(number)

528277

In [ ]:
for iteration in range(0, 100):
    newnumber = np.cumsum(number[::-1]) % 10
    number = newnumber[::-1]
digits = ''
digits = digits.join([str(x) for x in number])
print('The first eight digits are {0}.'.format(digits[0:8]))

In [229]:
number[0:8]

array([9, 1, 6, 8, 9, 3, 8, 0], dtype=int32)

In [199]:
for j in range(0, 100):
    newnumber = []
    print(j)
    for i in range(1, len(number)+1):
        pat = repeat_pattern(pattern, i, len(number))
        next_digit = sum(np.multiply(number, np.asarray(pat)))
        newnumber.append(int(str(next_digit)[-1]))
    number = np.array(deepcopy(newnumber))
digits = ''
digits = digits.join([str(x) for x in number])
print('The first eight digits are {0}.'.format(digits[int(digits[0:7]):int(digits[0:7])+8]))

0


KeyboardInterrupt: 

In [198]:
int(digits[0:7])

5867213

In [5]:
class moon_simulator(object):
    
    def __init__(self, coordinate_list):
        
        self.coordinate_list = coordinate_list
        self.velocity_list = np.zeros(coordinate_list.shape)
        self.number_planets = len(coordinate_list)
        
    def step(self):
        
        for i, coordinates in enumerate(self.coordinate_list):
            smaller = (coordinates < np.delete(self.coordinate_list, i, 0)).astype(int)
            equal = (coordinates == np.delete(self.coordinate_list, i, 0)).astype(int)
            # mapping: y/(num_pl-1) + 2*(num_pl-1) - (num_pl-1)
            delta_vel = sum(smaller)*2-(self.number_planets-1)+sum(equal)
            self.velocity_list[i] += delta_vel
            
        self.coordinate_list += self.velocity_list
        
    def energy(self):
        
        self.e_p = np.sum(abs(self.coordinate_list), 1)
        self.e_k = np.sum(abs(self.velocity_list), 1)
        self.e = np.sum(self.e_p*self.e_k)

In [6]:
steps = 1000
moons = moon_simulator(deepcopy(coordinate_list))
for i in range(0, steps):
    moons.step()

moons.energy()
print('The energy of the system after {0} steps is {1}.'.format(steps, moons.e))


The energy of the system after 1000 steps is 7179.0.


# Puzzle 2: N-body problem - calculate length of period

### all the axes are independent, so we can split the system; whole period is the least common multiple of all three periods

#### at first, set up a moon simulator for each axis and store the initial state

In [7]:
simulator_dict = dict()
states_init_dict = dict()
for i in range(0, len(coordinate_list[0])):
    simulator_dict['{0}'.format(i)] = moon_simulator(deepcopy(coordinate_list[:, i].reshape(-1, 1)))
    states_init_dict['{0}'.format(i)] = np.append(simulator_dict['{0}'.format(i)].coordinate_list,
                                                 simulator_dict['{0}'.format(i)].velocity_list)

#### now, simulate the systems until they reach their initial state and calculate the lcm

In [8]:
def lcm(values):
    multiple = values[0]
    for i in range(1, len(values)):
        multiple = multiple*values[i] // gcd(multiple, values[i])
    return multiple

In [9]:
periods = []
for key in simulator_dict.keys():
    state = np.zeros(states_init_dict[key].shape)
    period = 0
    while sum(states_init_dict[key] == state) != len(states_init_dict[key]):
        simulator_dict[key].step()
        state = np.append(simulator_dict[key].coordinate_list, simulator_dict[key].velocity_list)
        period += 1
    periods.append(period)
print('The period length of the system is {0}.'.format(lcm(periods)))

The period length of the system is 428576638953552.
